<a href="https://colab.research.google.com/github/vsunday/gcolab/blob/main/comp_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install signate

     |████████████████████████████████| 143kB 6.9MB/s 
  Created wheel for wget: filename=wget-3.2-cp37-none-any.whl size=9675 sha256=55c88e6c699905c341d116f2a45f3705cd30f8c117a7763a1d5d1b29cab63bc5
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget
ERROR: requests 2.23.0 has requirement urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you'll have urllib3 1.26.5 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [2]:
import os
os.environ['COMP_ID'] = '463'

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!mkdir /root/.signate
!cp /content/drive/MyDrive/'Colab Notebooks'/signate/signate.json /root/.signate
!mkdir /content/signate
%cd /content/signate
!signate download --competition-id $COMP_ID || ls

/content/signate
sample_submit.csv

test.csv

train.csv


Download completed.


In [4]:
import pandas as pd
#import xgboost as xgb
import numpy as np

from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedStratifiedKFold

In [5]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [6]:
train.head(10)

,id,age,sex,bmi,children,smoker,region,charges
0,0,26,male,32.665465,3,no,southeast,0
1,1,41,male,29.798725,1,no,southwest,0
2,2,28,male,32.722029,0,yes,northwest,1
3,3,20,female,38.429831,2,no,southeast,0
4,4,45,female,29.641854,1,no,northwest,0
5,5,20,male,37.785937,2,yes,southeast,1
6,6,37,male,43.393374,2,no,northeast,0
7,7,41,female,32.280407,1,no,northeast,0
8,8,40,male,28.865828,1,no,northeast,0
9,9,54,female,33.870887,2,no,northeast,0


In [14]:
def feature_engineering(data, reference, combination=True):
  def additional_features(data, combination):
    ref = data.copy(deep=True)
    # additional features
    ## bmi category
    bmi = ref['bmi']
    def make_bmi_category(bmi):
      category = [[18.5, 'underweight'], [25, 'normal'], [30, 'pre-obe'], [35, 'obe1'], [40, 'obe2']]
      for c in category:
        if bmi < c[0]:
          return c[1]
      return 'obe3'

    ref['bmi_category'] = bmi.apply(lambda x: make_bmi_category(x))

    columns = ['sex', 'children', 'smoker', 'region', 'bmi_category']
    
    ## feature combinations
    if combination:
      combinations = np.unique([np.sort([x,y]) for x in columns for y in columns if x!=y], axis=0)
      for [x,y] in combinations:
          ref[f'{x}_{y}'] = ref[x].apply(str).str.cat(ref[y].apply(str), sep='_')
    
    return ref

  data_added = additional_features(data, combination)
  reference_added = additional_features(reference, combination)

  def mean_encoding(data):
    category_columns = data.columns.drop(['id', 'age', 'bmi', 'charges']).to_list()
    return {k:data.groupby(k).mean()['charges'].to_dict() for k in category_columns}

  mec = mean_encoding(reference_added)

  def encode_normalize(data, mec):
    result = data.copy(deep=True)
    category_columns = data.columns.drop(['id', 'age', 'bmi', 'charges']).to_list() if 'charges' in data.columns else data.columns.drop(['id', 'age', 'bmi']).to_list()
    for c in category_columns:
      result[c] = result[c].map(mec[c])

    for c in ['age', 'bmi']:
      result[c] = (data[c] - data[c].mean()) / data[c].std()
    
    return result

  result = encode_normalize(data_added, mec)
  if 'charges' in result.columns:
    columns = result.columns.tolist()
    columns.remove('charges')
    all_columns = columns + ['charges']
    result = result[all_columns]
  return result

train_engd = feature_engineering(train,train,True)

In [9]:
train_engd.corr().style.background_gradient(cmap='coolwarm')

,id,age,sex,bmi,children,smoker,region,bmi_category,bmi_category_children,bmi_category_region,bmi_category_sex,bmi_category_smoker,children_region,children_sex,children_smoker,region_sex,region_smoker,sex_smoker,charges
id,1.000000,0.005803,-0.000223,0.039221,-0.025552,0.018419,0.040518,0.046413,0.023746,0.052913,0.036942,0.032072,0.002113,-0.017624,0.014082,0.035078,0.020861,0.018910,0.013463
age,0.005803,1.000000,0.016683,0.019885,-0.020654,-0.012406,-0.056213,0.029149,-0.000823,0.006358,0.031994,-0.018999,-0.051310,0.005812,0.008388,-0.025908,-0.008140,-0.011479,0.102112
sex,-0.000223,0.016683,1.000000,0.012805,-0.086868,0.170954,0.064807,0.013994,-0.022908,0.051037,0.432206,0.153101,0.008260,0.582105,0.158747,0.617270,0.171496,0.163655,0.118389
bmi,0.039221,0.019885,0.012805,1.000000,-0.020520,0.013406,0.235714,0.846628,0.615494,0.699417,0.665623,0.215961,0.150438,-0.007886,0.027122,0.189850,0.044765,0.013901,0.171282
children,-0.025552,-0.020654,-0.086868,-0.020520,1.000000,0.126653,0.017239,-0.028352,0.468531,-0.012438,-0.063777,0.120912,0.610126,0.678697,0.186048,-0.038394,0.119814,0.127736,0.138033
smoker,0.018419,-0.012406,0.170954,0.013406,0.126653,1.000000,0.127178,0.070951,0.175167,0.144981,0.153106,0.897239,0.180943,0.213636,0.974828,0.198683,0.983014,0.999786,0.723247
region,0.040518,-0.056213,0.064807,0.235714,0.017239,0.127178,1.000000,0.186517,0.154517,0.583519,0.190873,0.169924,0.667110,0.052278,0.134750,0.786913,0.205133,0.126840,0.150925
bmi_category,0.046413,0.029149,0.013994,0.846628,-0.028352,0.070951,0.186517,1.000000,0.726318,0.827306,0.781184,0.265457,0.123879,-0.005600,0.083642,0.157497,0.091535,0.071696,0.213980
bmi_category_children,0.023746,-0.000823,-0.022908,0.615494,0.468531,0.175167,0.154517,0.726318,1.000000,0.612357,0.541491,0.326563,0.403377,0.339503,0.197279,0.103246,0.188254,0.176271,0.294609
bmi_category_region,0.052913,0.006358,0.051037,0.699417,-0.012438,0.144981,0.583519,0.827306,0.612357,1.000000,0.677145,0.319301,0.386035,0.028218,0.156727,0.474623,0.185373,0.145547,0.258647


In [10]:
parameter_candidates = {
    'n_estimators': [10, 50, 100, 150, 200],
    'max_depth': [None, 5, 10, 20, 50, 100]
    }

In [11]:
X, y = train_engd.drop('charges',axis=1), train_engd['charges']

# cross validation
# clf = RandomForestClassifier(n_estimators=n)
clf = GridSearchCV(estimator=RandomForestClassifier(), param_grid=parameter_candidates, n_jobs=-1)
clf.fit(X, y)
# scores = cross_val_score(clf, X, y, cv=5)
# print(scores)
# print(scores.mean(), scores.std())

GridSearchCV(cv=None, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              ra

In [12]:
clf.best_score_

0.874375

In [15]:
data_test = feature_engineering(test, train)
preds_test = clf.predict(data_test)

In [17]:
from datetime import datetime


result = pd.concat([test["id"], pd.Series(preds_test)], axis=1)

filename = f'result_{datetime.today():%Y%m%d_%H%M}.csv'
result.to_csv(filename, header=False,index=False)

#from google.colab import files
#files.download(filename)

In [19]:
# submit
# !signate submit --competition-id 463 {filename}

You have successfully submitted your predictions.We will send you the submission result to your email address.
